In [21]:
!pip install sentence_transformers
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [37]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [38]:
import boto3

# Initialize Boto3 client
s3 = boto3.client('s3')

# Define S3 bucket and file key
bucket_name = 'seeyoul8rallidater'
file_key = 'okcupid_profiles.csv'

# Specify local file path to save the dataset
local_file_path = 'okcupid.csv'

# Download the dataset from S3 to local SageMaker Studio instance
s3.download_file(bucket_name, file_key, local_file_path)
df = pd.read_csv("okcupid.csv")
print("Dataset downloaded successfully!")


Dataset downloaded successfully!


In [39]:
df_sample = df.sample(n=10000, random_state = 13)
df_sample

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
34244,23,single,f,straight,used up,strictly anything,socially,never,working on college/university,"middle eastern, hispanic / latin, other",...,rather than writing an article about my life s...,i'm enjoying my youth while i still can and wo...,many many things.,oddly enough.. one of the first things people ...,"movies... girls interrupted, velvet goldmine, ...","friends, family, iphone, water, laughter, san ...",my future and how i can better myself.,out with some friends in the city clubbing or ...,... it wouldn't be private if i admitted it on...,"you want to, of course."
12899,30,single,m,straight,fit,anything,socially,never,college/university,hispanic / latin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19963,22,single,f,straight,athletic,NaN,socially,NaN,working on college/university,white,...,"hi! my name is tricia, i am currently a studen...",-school -working on apparel design degree - wo...,lots of things but i'm really really good at s...,NaN,my all time favorite book is fight club favori...,friends and family sewing machine the gym natu...,i spend a lot of time thinking about or reflec...,currently on a typical friday night i am windi...,NaN,you're cool we have a few things in common
25079,36,single,f,straight,average,other,often,never,graduated from masters program,hispanic / latin,...,i am a person that enjoys outdoor activities. ...,i am enjoying my life and every challenge that...,cooking and baking,my smile :),architecture books life is beauty music concer...,"love, dedication, organization, wine, chocolat...",today!,enjoying a nice dinner with red wine and relax...,everything but with the right person :),you are respectful!
59873,36,single,f,straight,average,mostly anything,socially,never,graduated from masters program,white,...,"so, i went to a dinner party a while ago, and ...","making the world a better place, one impressio...",having ideas. (implementation is less consiste...,recently i was in the parking lot before an a'...,"books: complications, david copperfield, mysti...",(in no particular order) 1. coffee in the morn...,...how fascinating children's minds are and ho...,trying to rally some co-workers to go to happy...,i can't stop myself from clicking on links to ...,"you're smart, funny, ambitious, and doing good..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7212,59,single,m,straight,average,vegetarian,rarely,never,graduated from college/university,white,...,lived in sf bay area most of my life. come fro...,working in human resources. lots of other thin...,"loving, laughing, living",better for others to decide.,"beatles, green day, georgia o'keeffe, paul cez...","children, sunshine, sleep, present awareness, ...",let me think about that one.,"relaxing, playing music, movie watching, readi...",do i have to say something here?,you're interested.
32534,19,single,m,straight,a little extra,NaN,not at all,never,graduated from high school,"pacific islander, white",...,i am a nice person to know. i am usually quiet...,well i graduated from high school and went to ...,"i am really good at being sarcastic, sports, g...",people usually notices how i look mexican but ...,well i honestly dont like to read books alot b...,family friends video games movies (preferab...,what i can do to help my family and friends ou...,usually chillin at home playing video games :),i've never been in a relationship and that i a...,you are interested or if you like a few things...
6627,29,single,m,straight,average,NaN,often,sometimes,graduated from college/university,white,...,im not sure about if i want to grow up. having...,trying to figure that out. not too anxious. i ...,"navigating, cooking, and gardening.",i can talk about anything. i usually can find ...,"music: i like 90's slacker indie rock, and psy...","my bicycle, the sea, my guitar

In [40]:
# All essays but essay9, which is "you should message me if..."

df_demographics = df_sample[df_sample.columns.drop(list(df_sample.filter(regex="essay")))]
essays_df = df_sample.loc[:, ["essay0", "essay1", "essay2", "essay3", "essay4", 
                   "essay5", "essay6", "essay7", "essay8"]]
essays_df = essays_df.fillna(" ").astype(str)

essays_df.loc[:, "all_essays"] = essays_df.apply(" ".join, axis = 1)

df_all = pd.concat([df_demographics, essays_df.loc[:, ["all_essays"]]], axis = 1)

In [41]:
for e in df_all.all_essays[:10]:
    print(e)
    print("NEW BIO")

rather than writing an article about my life story or analyzing every possible personality trait i have, i'm going to keep this to a minimal. i'm currently a student.. so school has been my main focus. but when i'm not busy with that, i definitely like to kick back and enjoy what life has to offer. i like going on road trips (random or not), discovering new restaurants, going to dive bars, shopping, going to shows and so on. i'm working on being more spontaneous and it's coming along fairly well. haha. i would like to think of myself as a humble, open-minded person who is very accepting of others.. as long as you respect me, my beliefs and opinions then i will do the same. i'm also extremely honest and a bit sarcastic so if you are someone who is easily offended, please keep that in mind.  the reason why i joined this site no reason in particular. i'm not on a quest to find my soul mate or look for a relationship. there is a time for everything in life and some things aren't meant to b

In [42]:
# Extract all essays from the DataFrame into a list
essays_list = df_all["all_essays"].tolist()

# Encode all essays using the model
embedding_list = model.encode(essays_list)

embedding_list = embedding_list.tolist()
# Create a Pandas Series with embeddings and indices
embedding_series = pd.Series(embedding_list, index=df_all.index, name="embedding")


In [43]:
embedding_series

34244    [0.035748451948165894, -0.041070356965065, 0.0...
12899    [-0.11883843690156937, 0.04829869419336319, -0...
19963    [-0.040667980909347534, -0.09981440007686615, ...
25079    [0.05029098317027092, -0.006492716260254383, 0...
59873    [0.033390577882528305, -0.08353191614151001, 0...
                               ...                        
7212     [0.09897657483816147, -0.029563361778855324, 0...
32534    [0.01558070071041584, -0.10636980086565018, 0....
6627     [0.06526684761047363, -0.1282835155725479, 0.0...
21102    [-0.032097410410642624, -0.06788887083530426, ...
35720    [-0.03524378687143326, -0.0545826256275177, -0...
Name: embedding, Length: 10000, dtype: object

In [44]:
embedding_series.to_csv('embedding_series.csv') 

export_bucket_name = 'seeyoul8rallidater'
export_file_name = 'embedding_series.csv'
s3.upload_file('embedding_series.csv', export_bucket_name, export_file_name)

embedding_series

34244    [0.035748451948165894, -0.041070356965065, 0.0...
12899    [-0.11883843690156937, 0.04829869419336319, -0...
19963    [-0.040667980909347534, -0.09981440007686615, ...
25079    [0.05029098317027092, -0.006492716260254383, 0...
59873    [0.033390577882528305, -0.08353191614151001, 0...
                               ...                        
7212     [0.09897657483816147, -0.029563361778855324, 0...
32534    [0.01558070071041584, -0.10636980086565018, 0....
6627     [0.06526684761047363, -0.1282835155725479, 0.0...
21102    [-0.032097410410642624, -0.06788887083530426, ...
35720    [-0.03524378687143326, -0.0545826256275177, -0...
Name: embedding, Length: 10000, dtype: object

In [45]:
print(df_all.loc[:,'sex'].unique())
print(df_all.loc[:,'orientation'].unique())


['f' 'm']
['straight' 'bisexual' 'gay']


In [46]:
df_all['pref_gen'] = np.where((df_all[ 'sex'] == 'm') & (df_all['orientation'] == 'gay'), 'm', 
                        np.where((df_all['sex'] == 'm') & (df_all['orientation'] == 'straight'), 'f',
                        np.where((df_all['sex'] == 'f') & (df_all['orientation'] == 'straight'), 'm',
                        np.where((df_all['sex'] == 'f') & (df_all['orientation'] == 'gay'), 'f',
                        np.where((df_all['orientation'] == 'bisexual'), 'all', 'no one')))))
print(df_all.loc[:,'pref_gen'].unique())

['m' 'f' 'all']


In [47]:
def compute_cosine_similarity(target_vector, vectors):
    similarities = []
    for vector in vectors:
        similarity = 1 - cosine(target_vector, vector)  # 1 - cosine distance to get cosine similarity
        similarities.append(similarity)
    return similarities

In [48]:
def rank_matches(input_row, pref_age_lower=False, pref_age_higher=False, min_similarity_score = 0.65):
    df_possible = df_all.copy()
    if pref_age_higher:
        df_possible = df_possible[df_possible['age'] <= df_possible.loc[input_row, 'age'] + pref_age_higher]
    if pref_age_lower:
        df_possible = df_possible[df_possible["age"] >= df_possible.loc[input_row,'age'] - pref_age_lower]
    if df_possible.loc[input_row,'pref_gen'] != 'all':
        df_possible = df_possible.loc[df_possible.loc[input_row,'pref_gen'] == df_possible.loc[:,'sex'],:]

    user_embeddings = embedding_series[input_row]

    other_embeddings = [embedding_series[i] for i in df_possible.index]
    # Compute the cosine similarity between the user's weighted embedding vector and all possible matches
    cosine_similarities = compute_cosine_similarity(user_embeddings, other_embeddings)
    # Recover index to match back to original dataframe
    similarity_scores = [(df_possible.index[index], score) for index, score in enumerate(cosine_similarities) if score >= min_similarity_score and score != 1]
    # Sort by similarity
    ranked_similarity = sorted(similarity_scores, key = lambda x: x[1], reverse = True)

    return ranked_similarity

In [49]:
test = rank_matches(34244, 10, 10)

In [50]:
test 

[(14811, 0.6980022164909875),
 (7520, 0.6977487961228467),
 (48089, 0.6911373638822215),
 (9239, 0.6787613231822924),
 (5324, 0.6780443014414438),
 (56246, 0.6759933644888927),
 (42074, 0.6740422185535244),
 (37288, 0.6731679989658562),
 (7408, 0.670816196547776),
 (56002, 0.6684090917929425),
 (6457, 0.668364088621221),
 (50943, 0.6589050617052672),
 (48484, 0.6557934530854321),
 (30074, 0.6555141575305452),
 (10023, 0.6553539395315249),
 (48407, 0.6535909315244728),
 (29700, 0.6521652921380251),
 (11614, 0.6520390983285584),
 (40848, 0.6519792920134156),
 (2820, 0.6504539897362995)]

In [ ]:
df_all['matches'] = [rank_matches(i,10,10) for i, row in df_all.iterrows()]


In [ ]:
df_all['matches']

34244    [(14811, 0.6980022164909875), (7520, 0.6977487...
12899    [(43433, 0.9999999999995626), (41254, 0.999999...
19963    [(43889, 0.7365690331058113), (1661, 0.7215441...
25079    [(49930, 0.7836977366844995), (27537, 0.764374...
59873    [(15635, 0.7134278683277475), (40848, 0.702959...
                               ...                        
7212                         [(18998, 0.6534555292817982)]
32534    [(9195, 0.6787860705914651), (39887, 0.6671304...
6627     [(21498, 0.7603978373936351), (46973, 0.758050...
21102                                                   []
35720                                                   []
Name: matches, Length: 10000, dtype: object

In [ ]:
df_all['matches'].to_csv('okcupid_matches.csv') 

export_bucket_name = 'seeyoul8rallidater'
export_file_name = 'okcupid_matches.csv'

# Upload the exported CSV file to S3
s3.upload_file('okcupid_matches.csv', export_bucket_name, export_file_name)

print("DataFrame exported to S3 successfully!")

DataFrame exported to S3 successfully!


In [ ]:
def rank_new_input(input_str, pref_gender=False, pref_age_lower=False, pref_age_higher=False, min_similarity_score = 0.5):
    df_possible = df_all.copy()
    if pref_gender:
        df_possible = df_possible.loc[df_possible.loc[:,'sex'] == pref_gender, :]
    if pref_age_higher:
        df_possible = df_possible[df_possible.loc[:, "age"] <= pref_age_higher]
    if pref_age_lower:
        df_possible = df_possible[df_possible.loc[:, "age"] >= pref_age_lower]
    user_embeddings = model.encode(input_str)
    other_embeddings = [embedding_series[i] for i in df_possible.index]
    
    # Compute the cosine similarity between the user's weighted embedding vector and all possible matches
    cosine_similarities = compute_cosine_similarity(user_embeddings, other_embeddings)
    # Recover index to match back to original dataframe
    similarity_scores = [(df_possible.index[index], score) for index, score in enumerate(cosine_similarities) if score >= min_similarity_score and score != 1]
    # Sort by similarity
    ranked_similarity = sorted(similarity_scores, key = lambda x: x[1], reverse = True)
    return ranked_similarity

In [ ]:
rank_new_input("I like bacon and pumpkin pie",'f', 30,40, 0.2)


[(57393, 0.39922267782587606),
 (19825, 0.3848119401395327),
 (23315, 0.3848033611374957),
 (53574, 0.36858424205535956),
 (34981, 0.367990870726431),
 (46035, 0.3649618384360964),
 (10869, 0.36213544891805927),
 (27126, 0.35618387778451654),
 (12023, 0.350264353822047),
 (37122, 0.3420075723344681),
 (46031, 0.3409200376529504),
 (56690, 0.33969013814134064),
 (59903, 0.32543778845263993),
 (56485, 0.3250035666564459),
 (36269, 0.3247590089049567),
 (58784, 0.32403927516788866),
 (19876, 0.3175724944269229),
 (24597, 0.3137599191194095),
 (30288, 0.31146275280251434),
 (18289, 0.3110462419800759),
 (52449, 0.30965161847462586),
 (10185, 0.30764708415096154),
 (8780, 0.30429316749318325),
 (23624, 0.29971042062028785),
 (31183, 0.2970314947698399),
 (56312, 0.2963319818669611),
 (25377, 0.2953194976154485),
 (22764, 0.2941509440621166),
 (59745, 0.2932016253945142),
 (9372, 0.28963554782873546),
 (54930, 0.28621548104805083),
 (33890, 0.28457696267840826),
 (35871, 0.2841369559386393),